### Code: Berezinskii-Kosterlitz-Thouless Renormalization Group Flow for Finite Size Scaling of the Bose-Hubbard Model
Authors


#### Abstract 
abstract

#### Requirements 
To run this code, Julia version 1.10.4 or higher is required. The cell below will install the required packages.

### Install packages

In [1]:
using Pkg 
Pkg.activate(".")

  Activating project at `C:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\Postdoc_UL\DelMaestroGroupRepositories\papers-code-BKTFlowBoseHubbard\src`


In [2]:
Pkg.add([
    "Plots", 
    "PyFormattedStrings", 
    "NonlinearSolve", 
    "StaticArrays", 
    "Printf", 
    "Integrals", 
    "FastClosures", 
    "LaTeXStrings", 
    "DataFrames",  
    "NPZ", 
    "Measures", 
    "PyCall"]);

   Resolving package versions...
  No Changes to `C:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\Postdoc_UL\DelMaestroGroupRepositories\papers-code-BKTFlowBoseHubbard\src\Project.toml`
  No Changes to `C:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\Postdoc_UL\DelMaestroGroupRepositories\papers-code-BKTFlowBoseHubbard\src\Manifest.toml`
Precompiling project...
   1485.3 ms  ✓ Expat_jll
   1506.8 ms  ✓ Xorg_libxcb_jll
   1678.1 ms  ✓ FreeType2_jll
   2160.5 ms  ✓ DifferentiationInterface → DifferentiationInterfaceForwardDiffExt
   2350.7 ms  ✓ Glib_jll
   2227.5 ms  ✓ ColorVectorSpace → SpecialFunctionsExt
   2267.3 ms  ✓ RecursiveArrayTools → RecursiveArrayToolsForwardDiffExt
   2327.3 ms  ✓ NLSolversBase
   2348.0 ms  ✓ ForwardDiff → ForwardDiffStaticArraysExt
    833.9 ms  ✓ Xorg_libXext_jll
   1421.1 ms  ✓ Fontconfig_jll
   1876.6 ms  ✓ LoopVectorization → SpecialFunctionsExt
   2218.5 ms  ✓ LoopVectorization → ForwardDiffExt
    841.8 ms  ✓ Libglvnd_jll
   2817.4 ms  ✓ L

### Import

In [3]:
using Plots
using PyFormattedStrings
using NonlinearSolve
using StaticArrays
using Printf
using Integrals
using FastClosures
using LaTeXStrings
using DataFrames 
using NPZ
using Measures
using PyCall

[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a] (cache misses: wrong dep version loaded (4), incompatible header (2))


### Include 

In [426]:
include("./plot_utils.jl"); 
include("./load.jl");
include("./fit.jl");
include("./rg.jl");
include("./fss.jl")
include("./plot_routines.jl");

### Figure 1: BKT RG Flow
RG flow close to the BKT phase transition constructed from the DMRG simulations via 
 
\begin{align*}
    2\ln\!\left(\frac{L_2}{L_1}\right)  &= \int_{K_2}^{K_1} \frac{{\rm d}K}{K^2 (\ln(K/2)-\zeta) + 2K} \\
            \zeta  &\equiv \frac{2}{K} + \ln\left(\frac{K}{2}\right) - \frac{\pi^2}{32} \left(\frac{V}{E_r}\right)^2  \ .
\end{align*}
 
Each line corresponds to one value of $\zeta(U)$ obtained from the largest two system sizes for each $U/J$. Here, $K$ is the Luttinger parameter and $V$ the strength of the cosine lattice potential and $E_r$ a characteristic energy scale. Circles show the points for individual system sizes (larger circles correspond to larger $L$, $L\in\{16,32,48,64,96,128\}$). The dashed black line is the separatrix marking the phase transition $\zeta(U_c)=1$. Open circles are obtained by fits of the $L\to\infty$ limit of 
 
\begin{align*}
    \mathcal{F}_{\rm{pbc}}(\ell; K, \alpha) &= 
    \frac{K}{2\pi^2}\ln{\left[1+\frac{\sin^2{(\pi\ell/L)}}{\sinh^2{(\pi\alpha/L)}}\right]}   
\end{align*}
 
to data obtained for infinite boundary conditions using the VUMPS algorithm [V. Zauner-Stauber, L. Vanderstraeten, M. T. Fishman,F. Verstraete, and J. Haegeman, Variational optimization algorithms for uniform matrix product states, Physical Review B 97, 045145 (2018)]. Crosses are obtained from QMC. We find excellent agreement of the DMRG data with the predicted RG flow and between the different methods.

In [401]:
begin  
    # color lookup
	Us = get_values_of_U(;path="../data/pbc/DMRG/")
	local col_lookup = get_color_lookup(Us)
	# plot options
	local l = grid(1, 2, widths=[0.8,0.2])
    local ms_lookup =  get_marker_size_lookup()


    # pbc dmrg
	local p1 = plot(; xlabel=L"$K$", ylabel=L"$\pi V/(4E_r)$", ylim=(-0.001,.3999), xlim=(1.645,2.0+.77), legend=nothing,foreground_color_legend =nothing, legendtitle = nothing, legendtitlefontsize=9, handlelength=8  ) 
    local Us, ζs, ζserr = plot_dmrg_rg_flow!(p1, col_lookup, ms_lookup ) 
	# avoid accidental overwrite for values required below
	__Us, __ζs, __ζserr = Us, ζs, ζserr
	# separatrix 
    local Ks_sep, Vs_sep = get_flow_curve(1.0)
    plot!(p1, Ks_sep[Ks_sep.>=2.0], Vs_sep[Ks_sep.>=2.0]; color=:black, linewidth=1.0, linestyle=:dash, label=nothing) 
	add_arrow!(p1,Ks_sep[Ks_sep.>2.0], Vs_sep[Ks_sep.>2.0], 0.28,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.>2.0], Vs_sep[Ks_sep.>2.0], 0.35,:black;debug_print=true)
	plot!(p1, Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0]; color=:black, linewidth=1.0, linestyle=:dot, label=nothing)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.1,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.2,:black;debug_print=true)
	add_arrow!(p1,Ks_sep[Ks_sep.<2.0], Vs_sep[Ks_sep.<2.0], 0.3,:black;debug_print=true)

	# vumps data 
	plot_vumps_rg_flow!(p1, col_lookup, ms_lookup )

	# qmc data 
	plot_qmc_rg_flow!(p1, col_lookup, ms_lookup )
	
	# manually draw legend 
	local p_leg = plot(; framestyle = nothing) 
	draw_legend!(p_leg, Us, ζs, col_lookup, ms_lookup, highlight_U = [3.275])	
    println(ζs)
	
	# add phase labels
	annotate!(p1, 2.0, 0.38, text(L"\textrm{MI}", 10, :vcenter, :hcenter))
	annotate!(p1, 2.625, 0.38, text(L"\textrm{SF}", 10, :vcenter, :hcenter)) 
	
	
	local p = plot(p1, p_leg, layout = l, size=(1.2*600,1.1*600/2), bottom_margin=[2mm 2mm], left_margin=[2mm -5mm] )  
	savefig(p,"../figures/001_BKT_RG_flow_Bose-Hubbard.pdf");
	savefig(p,"../figures/001_BKT_RG_flow_Bose-Hubbard.png");
	#p 
end

┌ Warning: Bisection method did not converge to tol=1e-8. This may be due to a very steep function close to the zero.
└ @ Main C:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\Postdoc_UL\DelMaestroGroupRepositories\papers-code-BKTFlowBoseHubbard\src\rg.jl:107


1693
0.2797898440240061
0.28


[1.243051666053975, 1.242649365066592, 1.2422469338446176, 1.2418443723118837, 1.2414416803921555, 1.2410388580091358, 1.2406359050864626, 1.240232821547709, 1.2398296073163866, 1.239426262315939, 1.2390227864697474, 1.238619179701126, 1.2382154419333287, 1.2378115730895407, 1.237407573092884, 1.2370034418664153, 1.2365991793331277, 1.2361947854159474, 1.2357902600377377, 1.2353856031212942, 1.2349808145893482, 1.2345758943645686, 1.2341708423695539, 1.2337656585268408, 1.2333603427589, 1.232954894988134, 1.2325493151368851, 1.2321436031274235, 1.2317377588819576, 1.2313317823226304, 1.2309256733715157, 1.2305194319506239, 1.2301130579818984, 1.2297065513872172, 1.2292999120883905, 1.2288931400071643, 1.2284862350652168, 1.22807919718416, 1.2276720262855396, 1.227264722290834, 1.226857285121457, 1.2264497146987527, 1.226042010944001, 1.2256341737784122, 1.2252262031231345, 1.224818098899243, 1.2244098610277492, 1.2240014894295967, 1.2235929840256623, 1.2231843447367554, 1.2227755714836

, 0.7407865667493806, 0.7401930808537338, 0.7395993326201051, 0.7390053218402192, 0.7384110483055616, 0.7378165118073754, 0.7372217121366723, 0.7366266490842218, 0.736031322440556, 0.7354357319959699, 0.7348398775405138, 0.7342437588640041, 0.7336473757560108, 0.7330507280058716, 0.7324538154026758, 0.7318566377352743, 0.7312591947922784, 0.7306614863620544, 0.7300635122327287, 0.7294652721921827, 0.7288667660280577, 0.7282679935277477, 0.7276689544784071, 0.7270696486669453, 0.7264700758800235, 0.7258702359040644, 0.7252701285252399, 0.7246697535294784, 0.7240691107024654, 0.7234681998296344, 0.7228670206961754, 0.7222655730870318, 0.7216638567868987, 0.7210618715802245, 0.720459617251208, 0.719857093583799, 0.7192543003617026, 0.7186512373683699, 0.7180479043870025, 0.7174443012005552, 0.7168404275917323, 0.716236283342985, 0.7156318682365135, 0.7150271820542669, 0.7144222245779452, 0.713816995588991, 0.7132114948685996, 0.712605722197708, 0.7119996773570063, 0.7113933601269249, 0.71

, 0.20776072070610815, 0.20689165586640654, 0.20602207167514627, 0.2051519675780581, 0.20428134302002796, 0.20341019744508657, 0.20253853029642493, 0.20166634101637057, 0.20079362904640755, 0.19992039382716104, 0.19904663479838622, 0.19817235139900605, 0.1972975430670598, 0.1964222092397273, 0.19554634935333992, 0.19466996284335092, 0.19379304914434137, 0.19291560769003574, 0.19203763791327588, 0.1911591392460456, 0.190280111119435, 0.18940055296366962, 0.18852046420810237, 0.1876398442811961, 0.18675869261053374, 0.18587700862281947, 0.18499479174386999, 0.1841120413986153, 0.18322875701108976, 0.1823449380044568, 0.18146058380096583, 0.18057569382198171, 0.17969026748798517, 0.17880430421853363, 0.1779178034323056, 0.17703076454707603, 0.17614318697970607, 0.1752550701461629, 0.17436641346150433, 0.17347721633987828, 0.1725874781945171, 0.17169719843775746, 0.17080637648100352, 0.16991501173475695, 0.16902310360859008, 0.1681306515111654, 0.16723765485022413, 0.16634411303257932, 0.1

...

┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\Nonlinear

┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\Nonlinear

┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\NonlinearSolve\82B4C\src\internal\linear_solve.jl:162
┌ Warning: Potential Rank Deficient Matrix Detected. Attempting to solve using Pivoted QR Factorization.
└ @ NonlinearSolve C:\Users\mtham\.julia\packages\Nonlinear

[0.9923416671231682, 0.9953039238236674, 0.9972827507232727, 0.9990333731947758, 0.9994324604738796, 0.9998366679239847, 1.0000405948135016, 1.0002457680236259, 1.0019653890093034, 1.0032382852927253, 1.0078419548066735, 1.0128613146672423, 1.018266015060332, 1.0240689666585217, 1.0302923999333553, 1.0369652662390398, 1.0441223791417678]


"C:\\Users\\mtham\\Dropbox\\Dokumente\\Universität\\Skripte\\Postdoc_UL\\DelMaestroGroupRepositories\\papers-code-BKTFlowBoseHubbard\\figures\\001_BKT_RG_flow_Bose-Hubbard.png"

### Figure 2: Finite Size Scaling
Finite size scaling of the Luttinger parameter as a function of the system size $L$ according to the RG flow. The circles depict the results obtained from high precision DMRG calculations with open boundary conditions. Dashed lines show fits of the corresponding scaling form to the data. We show the three scaling regimes (i) in the superfluid phase according to $K_{\rm SF} \sim K^* + \kappa_0 \left(\frac{L_0}{L}\right)^{2(K^*-2)}$ (upper panel), (ii) for the value of $U$ closest to the separatrix using $K_{\rm sep} \sim 2  + \frac{1}{1/\kappa_0 + \ln L/L_0}$ (center panel), and (iii) in the insulating phase according to $K_{\rm MI} \sim 1 + (1+\kappa_0)\left(\frac{L_0}{L}\right)^{8 (1-\zeta)}$ (bottom panel).

In [402]:
# requires to run the cell above first 
ζlookup = Dict{Float64,Float64}()
for (U,ζ) in zip(__Us,__ζs)
    ζlookup[U] = ζ
end 

In [403]:
begin 
	local Us = [3.2,3.275,3.3]  
	local Ls_pbc = [16,32,48,64,96]
    local Ls_obc = [128,200,256,330,460,512]  
	local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/"))
	local cols = [col_lookup[U] for U in Us]

	local xlim = (0.0,0.375)
    local ylim = (1.95001,2.27)
    
    
	local boff = 0mm #-7mm
    local p = plot(;ylabel=L"$K$",xlabel=L"$1/\ln(L)$",xlim=xlim,ylim=ylim, size=(1.2*300,1.0*250),bottom_margin=[boff boff 0mm]) #,yticks=([1.6,1.8,2.0],["1.60","1.80","2.00"]))

    # labels
    annotate!(p,0.172+0.003,2.25, text(latexstring("\\mathrm{OBC}"),:center,9))
    annotate!(p,0.2305+0.003,2.25, text(latexstring("\\mathrm{PBC}"),:center,9))
    
	local xoff=0.003
    local x_right = 0.26
    
    annotate!(p,x_right,2.06,text(latexstring(f"$U\\ \\ \\ \\ \\ \\ \\ \\zeta$"),9,:left )) 
    plot!(p,[x_right-0.015,x_right-0.005], [2.03,2.03];color= cols[1])
	annotate!(p,x_right,2.03,text(latexstring(f"${Us[1]:2.1f}\\ \\ \\ \\ \\ {ζlookup[Us[1]]:2.4f}$"),9,:left )) 
    plot!(p,[x_right-0.015,x_right-0.005], [2.00,2.00];color= cols[2])
    annotate!(p,x_right,2.00,text(latexstring(f"${Us[2]:2.3f}\\;\\  {ζlookup[Us[2]]:2.4f}$"),9,:left ))   
    plot!(p,[x_right-0.015,x_right-0.005], [1.97,1.97];color= cols[3])
    annotate!(p,x_right,1.97,text(latexstring(f"${Us[3]:2.1f}\\ \\ \\ \\  \\ {ζlookup[Us[3]]:2.4f}$"),9,:left ))

	# x-axis transformation: plot 1/log(L)
	x_trafo = (x) -> 1.0 ./ log.(x)

	for (U, (i_c,c)) in zip(Us,enumerate(cols))
        #PBC
        
		local Ks_lin = zeros(Float64,length(Ls_pbc))
		local Ks_lin_err = zeros(Float64,length(Ls_pbc))
		local as_lin = zeros(Float64,length(Ls_pbc))
		for (i,L) in enumerate(Ls_pbc)
		    local ls, Fs = load_dmrg_pbc(L,U) 
		    local xs = lin_factor_pbc(ls,L)
		    local idx = window(L) 
			(Ks_lin[i], as_lin[i]), Ks_lin_err[i], _ = fit_fluc_lin_xs_pbc_err(xs,Fs,L,idx) 
    end    
		local Ls_fit = [5:1:1000;2000;5000;10000; 100000; 1000000;10000000;100000000;1000000000;10000000000;100000000000;100000000000;Inf]
		if i_c == 2 
			# ~sepraratrix 
			local Ks = fss_separatrix(Ls_fit, (Ls_pbc[end],Ks_lin[end])) 
			plot!(p, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		elseif i_c == 1
			# superfluid  
			local Ks = fss_superfluid(Ls_fit, (Ls_pbc[end-1:end], Ks_lin[end-1:end]))
			plot!(p, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		elseif i_c == 3	
			# mott insulator  
			local Ks = fss_mott_insulator(Ls_fit, (Ls_pbc[end-1:end], Ks_lin[end-1:end]))
			plot!(p, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
		end 
		
    
        scatter!(p, x_trafo(Ls_pbc), Ks_lin,yerr=Ks_lin_err; marker=:circle, msw=0.6, ms=4, nice_points(c)...)
		scatter!(p, x_trafo(Ls_pbc), Ks_lin ; marker=:circle, msw=2, ms=5, nice_points(c)...)
    
        # OBC
        local Ks_lin = zeros(Float64,length(Ls_obc))
		local Ks_lin_err = zeros(Float64,length(Ls_obc))
		local as_lin = zeros(Float64,length(Ls_obc))
                
        local window_fun_obc = L->round(Int,L/8):round(Int,L/8+L/32) 	
        for (i,L) in enumerate(Ls_obc) 
		    local ls, Fs = load_dmrg_obc(L,U) 
		    local xs = lin_factor_pbc(ls,L)
		    local idx = window_fun_obc(L)
			(Ks_lin[i], as_lin[i]), Ks_lin_err[i], _ = fit_fluc_lin_xs_pbc_err(xs,Fs,L,idx, mask_err=(length(xs)÷8+length(xs)÷32):(length(xs)÷2-length(xs)÷16)) 
end  
        scatter!(p, x_trafo(Ls_obc), Ks_lin,yerr=Ks_lin_err; marker=:circle, msw=0.6, ms=4, m=:square, nice_points(c)...) 
        scatter!(p, x_trafo(Ls_obc), Ks_lin; marker=:circle, msw=2, ms=4, m=:square, nice_points(c)...)

	end
	
	savefig(p,"../figures/002_finite_size_scaling.pdf");
	savefig(p,"../figures/002_finite_size_scaling.svg");
	#p
end 

[2.165121690718643, 0.1470899524983646]
[1.2062444151115694, 0.9976014124764223]


"C:\\Users\\mtham\\Dropbox\\Dokumente\\Universität\\Skripte\\Postdoc_UL\\DelMaestroGroupRepositories\\papers-code-BKTFlowBoseHubbard\\figures\\002_finite_size_scaling.svg"

In [ ]:
## begin 
#	local Us = [3.0,3.275,3.5]  
#	local Ls = [16,32,48,64,96]
#	local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/"))
#	local cols = [col_lookup[U] for U in Us]
#
#	local xlim = (0.1,0.375)
#
#	# mott insulator  
#	local p3 = plot(;ylabel=L"$K$",xlabel=L"$1/\ln(L)$",xlim=xlim,yticks=([1.6,1.8,2.0],["1.60","1.80","2.00"]))
#	local xoff=0.01
#	annotate!(p3,0.12-xoff,1.89,text(latexstring(f"$U={Us[3]:2.1f}$"),10,:left)) 
#	annotate!(p3,0.12352-xoff,2.0,text(latexstring(f"$\\zeta={ζlookup[Us[3]]:2.4f}$"),10,:left)) 
#	# superfluid  
#	local p1 = plot(;ylabel=L"$K$",xlim=xlim,xticks=(xticks(p3)[1][1],[""]),yticks=([2.36,2.37,2.38]),ylim=(2.35,2.39))
#	annotate!(p1,0.120 -xoff,2.375,text(latexstring(f"$U={Us[1]:2.1f}$"),10,:left))
#	annotate!(p1,0.12352-xoff,2.383,text(latexstring(f"$\\zeta={ζlookup[Us[1]]:2.4f}$"),10,:left)) 
#	# ~sepraratrix 
#	local p2 = plot(;ylabel=L"$K$",xlim=xlim,xticks=(xticks(p3)[1][1],[""]),yticks=([2.10,2.14,2.18]) )
#	annotate!(p2,0.12-xoff,2.15,text(latexstring(f"$U={Us[2]:2.3f}$"),10,:left)) 
#	annotate!(p2,0.12352-xoff,2.18,text(latexstring(f"$\\zeta={ζlookup[Us[2]]:2.4f}$"),10,:left)) 
#
#	# x-axis transformation: plot 1/log(L)
#	x_trafo = (x) -> 1.0 ./ log.(x)
#
#	for (U, (i_c,c),pi) in zip(Us,enumerate(cols),(p1,p2,p3))
#		local Ks_lin = zeros(Float64,length(Ls))
#		local Ks_lin_err = zeros(Float64,length(Ls))
#		local as_lin = zeros(Float64,length(Ls))
#		for (i,L) in enumerate(Ls)
#		    local ls, Fs = load_dmrg_pbc(L,U) 
#		    local xs = lin_factor_pbc(ls,L)
#		    local idx = window(L) 
#			(Ks_lin[i], as_lin[i]), Ks_lin_err[i], _ = fit_fluc_lin_xs_pbc_err(xs,Fs,L,idx) 
#		end   
#
#		local Ls_fit = [5:1:1000;2000;5000;10000; 100000]
#		if i_c == 2 
#			# ~sepraratrix 
#			local Ks = fss_separatrix(Ls_fit, (Ls[end],Ks_lin[end])) 
#			plot!(pi, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
#		elseif i_c == 1
#			# superfluid  
#			local Ks = fss_superfluid(Ls_fit, (Ls[end-1:end], Ks_lin[end-1:end]))
#			plot!(pi, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
#		elseif i_c == 3	
#			# mott insulator  
#			local Ks = fss_mott_insulator(Ls_fit, (Ls[end-1:end], Ks_lin[end-1:end]))
#			plot!(pi, x_trafo(Ls_fit),Ks; linestyle=:dash, color=c)
#		end 
#		
#		scatter!(pi, x_trafo(Ls), Ks_lin,yerr=Ks_lin_err; marker=:circle, msw=2, ms=5, nice_points(c)...)
#
#	end
#	
#	# combine subplots
#	local l = grid(3, 1)
#	local boff = -7mm
#	local p = plot(p1,p2,p3, layout = l, size=(1.2*300,1.2*250),bottom_margin=[boff boff 0mm])
#
#	savefig(p,"../figures/002_finite_size_scaling.pdf");
#	savefig(p,"../figures/002_finite_size_scaling.svg");
#	#p
#end 

### Figure 3: Obtaining Luttinger Parameter from Flucutations
Extracting the Luttinger parameter $K$ from the particle number fluctuations for different interaction strengths $U/J$ and systems of length $L=96$. We fit 

\begin{align*}
    \mathcal{F}_{\rm{pbc}}(\ell; K, \alpha) &= 
    \frac{K}{2\pi^2}\ln{\left[1+\frac{\sin^2{(\pi\ell/L)}}{\sinh^2{(\pi\alpha/L)}}\right]}   \approx \frac{K}{\pi^2}\ln\left[\sin\left(\frac{\pi \ell}{L}\right)\right]+A
\end{align*}

to the periodic boundary condition DMRG data for interval sizes close to $\ell=L/2$.

In [404]:
begin  
	local Us = [3.2,3.275,3.3]   

    
    local ylim1=(1.05,1.2)
    local xlim1=(-0.05,0.003)
	local p1 = plot(;ylabel=L"$\mathcal{F}$ (periodic)", bottom_margin=-10mm,ylim=ylim1,xlim=xlim1) # legend(;pos=:bottomright)..., 
	
    # inset
    local insetbox = bbox(0.001,0.002,0.3,0.398)
    pin1 = plot!(p1;
             ylabel=L"\mathcal{F}", 
             inset_subplots=(1, insetbox), 
             subplot=2,
             ymirror = true, 
             xtickfontsize=9, 
             ytickfontsize=9, 
             yticks=([0.6,1.0],["0.6","1.0"]),
             xticks=([-0.3,-0.1]),
             xlim=(-0.37,0.02),
             ylim=(0.3,1.3),
             framestyle=:semi  ) #  framestyle=:semi, :tickfont => (9, :black), xguidefontsize=9, yguidefontsize=9,ylim=(-0.5,7))  
  
    local ylim2=(1.250001,1.55)
    local xlim2=(-0.11,0.003)
    local p2 = plot(;ylabel=L"$\mathcal{F}$ (open)",xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylim=ylim2,xlim=xlim2 , top_margin=-4mm, bottom_margin=-5mm ) # legend(;pos=:bottomright)..., 
    pin2 = plot!(p2;
         ylabel=L"\mathcal{F}", 
         inset_subplots=(1, insetbox), 
         subplot=2,
         ymirror = true, 
         xtickfontsize=9, 
         ytickfontsize=9, 
         yticks=([0.6,1.0,1.4],["0.6","1.0","1.4"]),
         xticks=([-0.3,-0.1]),
         xlim=(-0.48,0.05),
         ylim=(0.4,1.7),
         framestyle=:semi ) #  framestyle=:semi, :tickfont => (9, :black), xguidefontsize=9, yguidefontsize=9,ylim=(-0.5,7))  
       
    
	local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/")) 
	local cols = [col_lookup[U] for U in Us]

	for (U, (i_c,c)) in zip(Us,enumerate(cols)) 
        # PBC
        
		local L = 96
		local ls, Fs = load_dmrg_pbc(L,U);
		local xs = lin_factor_pbc(ls, L);  
		local idx = window(L)
		local (Kfit, afit), Kerr ,  _ = fit_fluc_lin_xs_pbc_err(xs, Fs, L, idx)   
		 
		scatter!(p1, xs,Fs; marker=:circle, msw=2, ms=5, label=latexstring(f"\\ U={U:3.3f}"), nice_points(c)...)
		 
		plot!(p1, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dash,label=nothing)    
		plot!(p1, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing,linewidth=2)   

		# legend:
		scatter!(p1,[-0.015],[1.095-0.015*(i_c-1)];ms=5,nice_points(c)...)
		annotate!(p1, -0.015,1.095-0.015*(i_c-1),text(latexstring(f"\\ \\ \\ U={U:3.3f}"),:left,10),subplot=1)
            
        # inset
        scatter!(pin1, xs,Fs; marker=:circle, msw=2, ms=2, label=latexstring(f"\\ U={U:3.3f}"), nice_points(c)..., subplot=2)
		plot!(pin1, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dot,label=nothing, subplot=2)    
		plot!(pin1, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing,linewidth=2, subplot=2)   

      
        
        # OBC 
        local window_fun_obc = L->round(Int,L/8):round(Int,L/8+L/32)  
        local L = 460
		local ls, Fs = load_dmrg_obc(L,U);
		local xs = lin_factor_pbc(ls, L);  
		local idx = window_fun_obc(L)
		local (Kfit, afit), Kerr ,  _ = fit_fluc_lin_xs_pbc_err(xs, Fs, L, idx, mask_err=(length(xs)÷8+length(xs)÷32):(length(xs)÷2-length(xs)÷16))   
		 
		scatter!(p2, xs,Fs; marker=:circle, msw=2, ms=5, label=latexstring(f"\\ U={U:3.3f}"), nice_points(c)...)
		 
		plot!(p2, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dash,label=nothing)    
		plot!(p2, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing,linewidth=2)   
 
        # inset
        scatter!(pin2, xs,Fs; marker=:circle, msw=2, ms=2, label=latexstring(f"\\ U={U:3.3f}"), nice_points(c)..., subplot=2)
		plot!(pin2, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dot,label=nothing, subplot=2)    
		plot!(pin2, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing,linewidth=2, subplot=2)   

      
        
	end
    
    annotate!(p1,-0.027,1.065,text(L"L=96",10,:left),subplot=1)
    annotate!(p2,-0.06,1.28,text(L"L=460",10,:left),subplot=1)
    
    
    plot!(pin1, [xlim1[1],xlim1[2],xlim1[2],xlim1[1],xlim1[1]], [ylim1[1],ylim1[1],ylim1[2],ylim1[2],ylim1[1]]; color=:black, subplot=2, lw=0.3, ls=:dot) 
    plot!(pin2, [xlim2[1],xlim2[2],xlim2[2],xlim2[1],xlim2[1]], [ylim2[1],ylim2[1],ylim2[2],ylim2[2],ylim2[1]]; color=:black, subplot=2, lw=0.3, ls=:dot) 
    
     
	local l = grid(2, 1)
	local p = plot(p1,p2, layout = l, size=(1.2*300,2.2*200), bottom_margin=[0mm -3mm])

	savefig(p,"../figures/003_fit_K_to_fluctuations.pdf");
	savefig(p,"../figures/003_fit_K_to_fluctuations.svg");
	#p
end

"C:\\Users\\mtham\\Dropbox\\Dokumente\\Universität\\Skripte\\Postdoc_UL\\DelMaestroGroupRepositories\\papers-code-BKTFlowBoseHubbard\\figures\\003_fit_K_to_fluctuations.svg"

In [ ]:
#begin  
#	local Us = [3.000,3.275,3.5]
#
#
#	local p1 = plot(;ylabel=L"$\mathcal{F}$",xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$", ) # legend(;pos=:bottomright)..., 
#	  
#	local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/")) 
#	local cols = [col_lookup[U] for U in Us]
#
#	for (U, (i_c,c)) in zip(Us,enumerate(cols)) 
#
#		local L = 96
#		local ls, Fs = load_dmrg_pbc(L,U);
#		local xs = lin_factor_pbc(ls, L);  
#		local idx = window(L)
#		local (Kfit, afit), Kerr ,  _ = fit_fluc_lin_xs_pbc_err(xs, Fs, L, idx)   
#		 
#		scatter!(p1, xs,Fs; marker=:circle, msw=2, ms=5, label=latexstring(f"\\ U={U:3.3f}"), nice_points(c)...)
#		 
#		plot!(p1, xs[5:end], fluctuations_pbc_large_L_xs(xs[5:end], (Kfit, afit), (L,)),color=:black,ls=:dash,label=nothing)    
#		plot!(p1, xs[idx], fluctuations_pbc_large_L_xs(xs[idx], (Kfit, afit), (L,)),color=:black,label=nothing)   
#
#		# legend:
#		scatter!(p1,[-0.12],[0.62-0.09*(i_c-1)];ms=5,nice_points(c)...)
#		annotate!(-0.12,0.62-0.09*(i_c-1),text(latexstring(f"\\ \\ \\ U={U:3.3f}"),:left,10))
#  
#	end
#	local l = grid(1, 1)
#	local p = plot(p1, layout = l, size=(1.2*300,1.2*200), bottom_margin=[1mm 0mm])
#
#	savefig(p,"../figures/003_fit_K_to_fluctuations.pdf");
#	savefig(p,"../figures/003_fit_K_to_fluctuations.svg");
#	#p
#end

#### Figure 4: Extracting Crtical Point
Dependence of the flow parameter $\zeta$ on the interaction strength $U/J$ (blue circles). At the critical point $\zeta=1$, and $\zeta(U)$ is linear close to the phase transition. A linear fit (orange line) reveals  the critical point $U_c/J=3.276\pm0.001$. The fit is performed to the region shown by the solid orange line and we extend the fit with a dashed line to show deviations away from $U_c$.

In [425]:
begin  
	col_points = "#0C5DA5"
	col_fit = "#FFB347" 
	col_Uc = "#FF6961"

	local p = plot(;xlabel=L"U/J",ylabel=L"\zeta",size=(1.2*315,1.2*225)) 
	# fit linear function to points close to Uc
	local lin_fun(U,(Uc,α),_) = α*(U .- Uc) .+ 1.0
	local mask = 4:7
	local Uc, dUc, invUc, dinvUc, α, dα = py_fit_lin_err(__Us[mask],__ζs[mask],__ζserr[mask])  
	# plot data and fit
	scatter!(p,__Us,__ζs,yerr=__ζserr;ylim=(0.992,1.00999),xlim=(3.05,3.45),nice_points(col_points)...,ms=4)
	plot!(p,__Us[mask],lin_fun(__Us[mask],(Uc,α),());  color=col_fit ) 
	plot!(p,__Us,lin_fun(__Us,(Uc,α),()); ls=:dash, color=col_fit) 
	# put critial point as text
	text1 = latexstring(f"$U_c/J=\\,\\,\\, \\, \\,{Uc:4.3f}\\pm{dUc:3.3f}$")  
	text2 = latexstring(f"$J/U_c={invUc:4.4f}\\pm{dinvUc:3.4f}$") 
	annotate!(3.07,0.9945,text(text1,:left,:bottom,10))
	annotate!(3.07,0.9925,text(text2,:left,:bottom,10))

	plot!(p,[Uc,Uc],[0.999,1.001],legend=nothing,color=col_Uc)
	plot!(p,[Uc,Uc],[0.999,1.001],ribbon=dUc,legend=nothing,color=col_Uc)
	
	# GP inset 
	U_gp,z_gp,dz_gp = load_GP_z(;path="../data/pbc/GP/sGP_Data.txt")   
	
	local insetbox = bbox(0.598,0.002,0.4,0.398)
	plot!(p;xlabel=L"U/J",ylabel=L" 10^{5}\times(1-\zeta)^2", inset_subplots=(1, insetbox), subplot=2, yticks=([0.0,5.0],["0.0","5.0"]), xticks=([3.15,3.35]),xlim=(3.05,3.45),framestyle=:semi, :tickfont => (9, :black), xguidefontsize=9, yguidefontsize=9,ylim=(-0.5,7)) # ,ylabel=L"\zeta"
		
	plot!(p,[Uc,Uc],[-0.45,1/2],legend=nothing,color=col_Uc, subplot=2)
	plot!(p,[Uc,Uc],[-0.45,1/2],ribbon=dUc,legend=nothing,color=col_Uc, subplot=2)
	
	plot!(p,U_gp,z_gp*1e5,ribbon=dz_gp*1e5,fillalpha=.8, subplot=2, color=col_fit) 
	in_sGP = 12 .- [1,2,4,5,8,9,10]
	not_in_sGP = 12 .- [3,6,7] 
    println(__Us[not_in_sGP])
	scatter!(p,__Us[in_sGP],(__ζs[in_sGP] .- 1).^2*1e5;nice_points(col_points)...,ms=2, subplot=2)
	scatter!(p,__Us[not_in_sGP],(__ζs[not_in_sGP] .- 1).^2*1e5;nice_points(col_points)...,ms=2,marker=:x,markerstrokewidth=2, subplot=2)

	savefig(p,"../figures/004_zeta_of_U.pdf");
	savefig(p,"../figures/004_zeta_of_U.svg");
	#p
end

[3.225, 3.28, 3.29]


"C:\\Users\\mtham\\Dropbox\\Dokumente\\Universität\\Skripte\\Postdoc_UL\\DelMaestroGroupRepositories\\papers-code-BKTFlowBoseHubbard\\figures\\004_zeta_of_U.svg"

## Supplementary Material

### Figure S1: Difference between open and periodic boundary conditions

In [429]:
begin 
    local col_lookup = get_color_lookup(get_values_of_U(;path="../data/pbc/DMRG/")) 

    local p_pbc = plot(;axis=:off,size=(1.2*200,1.2*160),xlim=(-1.6,1.2),ylim=(-1.25,1.4), bottom_margin=[2mm 2mm] , top_margin=[2mm 2mm] )
    draw_periodic_lattice!(p_pbc ; n=13, nice_points("#000000")... , mswidth  =1 )
    
    local p_obc = plot(;axis=:off,size=(1.2*300,1.2*160), ylim=(-0.5,0.5),xlim=(-1,10.5))
    draw_open_lattice!(p_obc; nice_points("#000000")... , mswidth  =1 ) 

    Us = [3.0,3.1,3.2,3.3]
    # fluctuations 
    window_fun_obc = L->round(Int,L/8):round(Int,L/8+L/32)  
    # pbc 
    local p_F_pbc = plot(;xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylabel=L"$\mathcal{F}$ (periodic)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm], xlim=(-0.2,0.02) ,ylim=(0.7,1.3))
    plot_fluctuations_supplement!(p_F_pbc, Us, 96, col_lookup; load_fun=load_dmrg_pbc)
    annotate!(p_F_pbc,-0.18,1.25,text(latexstring("L=96"),11,:left))

    # obc 
    local p_F_obc = plot(;xlabel=L"$\frac{1}{\pi^2}\,\ln\sin\frac{\pi \ell}{L}$",ylabel=L"$\mathcal{F}$ (open)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(-0.2,0.02) ,ylim=(1.05,1.65))
    plot_fluctuations_supplement!(p_F_obc, Us, 460, col_lookup; load_fun=load_dmrg_obc, legend=false,window_fun=window_fun_obc, first_point=1,first_fit_value=30,mask_err=(xs)->length(xs)÷8+length(xs)÷32:length(xs)÷2-length(xs)÷16)
    annotate!(p_F_obc,-0.18,1.6,text(latexstring("L=460"),11,:left))

    
    # K(L)
    x_trafo = (x) -> 1.0 ./ log.(x)
    # pbc 
    local p_K_pbc = plot(;xlabel=L"$1/\ln(L)$",ylabel=L"$K$ (periodic)",size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm] , xlim=(0.17,0.4))
    plot_K_supplement!(p_K_pbc, Us, col_lookup; load_fun=load_dmrg_pbc, L_lookup=U->get_values_of_L("../data/pbc/DMRG/",U) )
    # obc 
    local p_K_obc = plot(;xlabel=L"$1/\ln(L)$",ylabel=L"$K$ (open)", size=(1.2*315,1.2*225), bottom_margin=[2mm 2mm],xlim=(0.14,0.22))
    plot_K_supplement!(p_K_obc, Us, col_lookup; load_fun=load_dmrg_obc ,L_lookup=U->get_values_of_L_obc("../data/obc/DMRG/",U),window_fun=window_fun_obc,mask_err=(xs)->length(xs)÷8+length(xs)÷32:length(xs)÷2-length(xs)÷16)

    # combine plots 
    local l = grid(3, 2; heights=[0.15,0.425,0.425], widths=[0.5,0.5 ])

    local p = plot(p_pbc,p_obc, p_F_pbc, p_F_obc, p_K_pbc, p_K_obc,layout=l, size=2.0.*(1.0*315,1.0*325), bottom_margin=[-10mm -10mm 2mm 2mm 2mm 2mm], top_margin = [0mm 0mm 0mm 0mm 0mm 0mm], left_margin=[2mm 2mm])

    savefig(p,"../figures/S001_compare_pbc_obc.pdf");
    savefig(p,"../figures/S001_compare_pbc_obc.svg");
end

┌ Warning: Unused font arg: color (typeof(color))
└ @ Plots C:\Users\mtham\.julia\packages\Plots\du2dt\src\components.jl:280
┌ Warning: Unused font arg: color (typeof(color))
└ @ Plots C:\Users\mtham\.julia\packages\Plots\du2dt\src\components.jl:280
┌ Warning: Unused font arg: color (typeof(color))
└ @ Plots C:\Users\mtham\.julia\packages\Plots\du2dt\src\components.jl:280
┌ Warning: Unused font arg: color (typeof(color))
└ @ Plots C:\Users\mtham\.julia\packages\Plots\du2dt\src\components.jl:280


"C:\\Users\\mtham\\Dropbox\\Dokumente\\Universität\\Skripte\\Postdoc_UL\\DelMaestroGroupRepositories\\papers-code-BKTFlowBoseHubbard\\figures\\S001_compare_pbc_obc.svg"

###  Reproduce fitting method and finite size scaling from literatue

In [428]:
begin
    local L = 128
    local U = 3.33333
    
    # lattice schematic
    local p_obc = plot(;axis=:off,size=(1.2*160,1.2*300), ylim=(-0.5,10.5),xlim=(-0.5,0.5),left_margin=-11mm,right_margin=-4mm,bottom_margin=-15mm)
    draw_open_lattice_horizontal!(p_obc; nice_points("#000000")... , mswidth  =1 )  
    
    # fitting method
    local p_fit_obc = plot(;xlabel=L"$L/w$",ylabel=L"$K$ (open)",size=(1.2*315,1.2*225), xlim=(.5,8.5) ,ylim=(1.90,2.07) )
    Kfit,Kerr = plot_fitting_method_literature!(p_fit_obc, U, L;colors=["#000000","#377a30","#d62728"])
    annotate!(2.5,1.98,text(latexstring(f"K={Kfit:3.2f}\\pm{Kerr:3.2f}"),10,:left))
    annotate!(2,1.945,text(L"\mathrm{center}\ \mathrm{window} ",10,:left))
        scatter!(p_fit_obc,[1.6],[1.944];m=:circle,nice_points("#000000")...)
    annotate!(2,1.93,text(L"$\mathrm{left}\ \mathrm{window}\ r=16$",10,:left))
        scatter!(p_fit_obc,[1.6],[1.93];m=:diamond,nice_points("#377a30")...)
    annotate!(2,1.915,text(L"$\mathrm{left}\ \mathrm{window}\ r=4$",10,:left))
        scatter!(p_fit_obc,[1.6],[1.915];m=:square,nice_points("#d62728")...) 
    
    # finite size scaling in literature
    local U = 3.3456
    local p_fss_obc = plot(;xlabel=L"$1/\ln L$",ylabel=L"$K$ (open)",size=(1.2*315,1.2*225), xlim=(-0.01,0.299) ,ylim=(1.98,2.042))
    local Ls_lit = [128,256,512] 
    local Ls_more = [64,96,180,200,220,240,260,280,330,380,420,460] 
    plot_fss_method_literature!(p_fss_obc, U, Ls_lit, Ls_more; colors=["#779ECB","#FF6961"])
    
    # combine plots 
    widhts = [0.2,1,1]
    local l = grid(1, 3; widths= widhts / sum(widhts))
    
    local p = plot(p_obc,p_fit_obc,p_fss_obc,layout=l, size= 2.0.*(1.0*315,1.0*325/2.7), bottom_margin=[-10mm -10mm 2mm ], top_margin = [0mm 0mm 0mm])

    savefig(p,"../figures/S002_comparison_obc_literature.pdf"); 
end

┌ Warning: Unused font arg: color (typeof(color))
└ @ Plots C:\Users\mtham\.julia\packages\Plots\du2dt\src\components.jl:280
┌ Warning: Unused font arg: color (typeof(color))
└ @ Plots C:\Users\mtham\.julia\packages\Plots\du2dt\src\components.jl:280


Esimated K:2.0039154074349588 ± 0.14786944675715616


"C:\\Users\\mtham\\Dropbox\\Dokumente\\Universität\\Skripte\\Postdoc_UL\\DelMaestroGroupRepositories\\papers-code-BKTFlowBoseHubbard\\figures\\S002_comparison_obc_literature.pdf"

In [427]:
begin
    local L = 128
    local U = 3.33333
    
    # lattice schematic
    local p_obc = plot(;axis=:off,size=(1.2*160,1.2*300), ylim=(-0.5,10.5),xlim=(-0.5,0.5),left_margin=-11mm,right_margin=-4mm,bottom_margin=-15mm)
    draw_open_lattice_horizontal!(p_obc; nice_points("#000000")... , mswidth  =1 )  
    
    # fitting method
    local p_fit_obc = plot(;xlabel=L"$L/w$",ylabel=L"$K$ (open)",size=(1.2*315,1.2*225), xlim=(.5,8.5) ,ylim=(1.90,2.07) )
    Kfit,Kerr = plot_fitting_method_literature!(p_fit_obc, U, L;colors=["#000000","#377a30","#d62728"])
    annotate!(2.5,1.98,text(latexstring(f"K={Kfit:3.2f}\\pm{Kerr:3.2f}"),10,:left))
    annotate!(2,1.945,text(L"\mathrm{center}\ \mathrm{window} ",10,:left))
        scatter!(p_fit_obc,[1.6],[1.944];m=:circle,nice_points("#000000")...)
    annotate!(2,1.93,text(L"$\mathrm{left}\ \mathrm{window}\ r=16$",10,:left))
        scatter!(p_fit_obc,[1.6],[1.93];m=:diamond,nice_points("#377a30")...)
    annotate!(2,1.915,text(L"$\mathrm{left}\ \mathrm{window}\ r=4$",10,:left))
        scatter!(p_fit_obc,[1.6],[1.915];m=:square,nice_points("#d62728")...) 
    
    # finite size scaling in literature
    local U = 3.3456
    local p_fss_obc = plot(;xlabel=L"$1/\ln L$",ylabel=L"$K$ (open)",size=(1.2*315,1.2*225), xlim=(-0.01,0.299) ,ylim=(1.98,2.042))
    local Ls_lit = [128,256,512] 
    local Ls_more = []#[64,96,180,200,220,240,260,280,330,380,420,460] 
    plot_fss_method_literature!(p_fss_obc, U, Ls_lit, Ls_more; colors=["#779ECB","#FF6961"])
    
    # combine plots 
    widhts = [0.2,1,1]
    local l = grid(1, 3; widths= widhts / sum(widhts))
    
    local p = plot(p_obc,p_fit_obc,p_fss_obc,layout=l, size= 2.0.*(1.0*315,1.0*325/2.7), bottom_margin=[-10mm -10mm 2mm ], top_margin = [0mm 0mm 0mm])

    savefig(p,"../figures/comparison_obc_literature.pdf"); 
end

┌ Warning: Unused font arg: color (typeof(color))
└ @ Plots C:\Users\mtham\.julia\packages\Plots\du2dt\src\components.jl:280
┌ Warning: Unused font arg: color (typeof(color))
└ @ Plots C:\Users\mtham\.julia\packages\Plots\du2dt\src\components.jl:280


Esimated K:2.0039154074349588 ± 0.14786944675715616


"C:\\Users\\mtham\\Dropbox\\Dokumente\\Universität\\Skripte\\Postdoc_UL\\DelMaestroGroupRepositories\\papers-code-BKTFlowBoseHubbard\\figures\\comparison_obc_literature.pdf"